In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
egitimVeri=pd.read_csv("C:/Users/omerk/Documents/GitHub/yapayzeka/hafta13/train.csv")
testVeri=pd.read_csv("C:/Users/omerk/Documents/GitHub/yapayzeka/hafta13/test.csv")